In [ ]:
pip install audioread


In [ ]:
import requests
import tempfile
import librosa

# URL of the audio file
url = 'https://112.124.48.213:443/file/audio/124d5a90-081c-b7dd-9e47-778f0b1a091f.wav'

# Download the audio file with SSL verification disabled
response = requests.get(url, verify=False)

# Create a temporary file and write the audio content to it
with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as temp_file:
    temp_file.write(response.content)
    temp_file_path = temp_file.name  # Get the path of the temporary file

# Load the audio using librosa
y, sr = librosa.load(temp_file_path, sr=None)

# Now y contains the audio signal, and sr is the sample rate
print(f"Audio data shape: {y.shape}, Sample rate: {sr}")


In [15]:
# Loading the libraries
from datasets import load_dataset
from transformers import pipeline
from funasr import AutoModel
from funasr.utils.postprocess_utils import rich_transcription_postprocess
import librosa
import numpy as np
import sounddevice as sd
from IPython.display import Audio as IPythonAudio

In [ ]:
import librosa
import sounddevice as sd
import scipy.io.wavfile as wav

def load_audio(file_path):
    # Load the audio file with the original sample rate
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    return audio_data, sample_rate

def play_audio(audio_data, sample_rate):
    # Play the audio data
    sd.play(audio_data, sample_rate)
    sd.wait()  # Wait until the sound has finished playing

def save_audio(audio_data, sample_rate, save_path):
    # Save the audio data as a .wav file using scipy
    # Convert the floating-point audio to 16-bit PCM
    audio_data_int16 = (audio_data * 32767).astype('int16')
    wav.write(save_path, sample_rate, audio_data_int16)
    print(f"Audio saved to {save_path}")

# Example usage
file_path = r'C:\Users\Administrator\Desktop\Backend-Algorithm-LLM\Audio_transcription_files\tmp_5_no7xo.wav'
save_path = r'C:\Users\Administrator\Desktop\Backend-Algorithm-LLM\Audio_transcription_files\output_audio.wav'

audio_data, sample_rate = load_audio(file_path)

# Save the audio to a new .wav file
save_audio(audio_data, sample_rate, save_path)

# Play the audio from the saved file
play_audio(audio_data, sample_rate)


In [ ]:
file_path = r'C:\Users\Administrator\Desktop\Backend-Algorithm-LLM\Audio_transcription_files\h.wav'
audio_data, sample_rate = librosa.load(file_path, sr=None)

In [ ]:
import soundfile as sf

# Load the file
file_path = r'C:\Users\Administrator\Desktop\Backend-Algorithm-LLM\Audio_transcription_files\output_audio.wav'

# Open the audio file to get the details
with sf.SoundFile(file_path) as file:
    # Bit depth is part of the file's format attributes
    bit_depth = file.subtype  # This gives the bit depth and encoding format
    sample_rate = file.samplerate
    channels = file.channels

    print(f"Bit Depth: {bit_depth}")
    print(f"Sample Rate: {sample_rate}")
    print(f"Channels: {channels}")


In [ ]:
# Determine the number of channels
if audio_data.ndim == 1:
    print("Mono audio (1 channel)")
elif audio_data.ndim == 2:
    num_channels = audio_data.shape[0]
    print(f"Stereo or multi-channel audio with {num_channels} channels")

In [ ]:
def load_audio(file_path):
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    return audio_data, sample_rate

def play_audio(audio_data, sample_rate):
    sd.play(audio_data, sample_rate)
    sd.wait()  # Wait until the sound has finished playing

# Example usage
file_path = r'C:\Users\Administrator\Desktop\Backend-Algorithm-LLM\Audio_transcription_files\test\tmp1gjsg3lv.mp3'
audio_data, sample_rate = load_audio(file_path)
play_audio(audio_data, sample_rate)


In [ ]:
# Print details
print("Sample Rate:", sample_rate)
print("Audio Array:", audio_array)
print("Audio Array datatype is Array: ", isinstance(audio_array,np.ndarray) )

In [ ]:
# Resampling the Audio so it matches to the requirement of the model
audio_16KHz = librosa.resample(audio_array,
                               orig_sr=sample_rate,
                               target_sr=16000)

In [ ]:
model_dir = "FunAudioLLM/SenseVoiceSmall"

# SenseVoiceSmall 


In [ ]:
# model_dir = r'C:/Users/Administrator/Desktop/LLM_work/SenseVoiceSmall'
# pretrained_model_path=  r'C:\Users\Administrator\Desktop\LLM_work\SenseVoiceSmall\model.pt'
model = AutoModel(
    model='iic/SenseVoiceSmall',
    # init_param = pretrained_model_path
)

# en
res = model.generate(
    input=audio_16KHz,
    cache={},
    language="auto",  # "zn", "en", "yue", "ja", "ko", "nospeech"
    use_itn=True,
    batch_size_s=60,
    merge_vad=True,  #
    merge_length_s=15,
)
text = rich_transcription_postprocess(res[0]["text"])
print(text)

# paraformer-zh

In [ ]:
audio_path = r'C:\Users\Administrator\Desktop\Backend-Algorithm-LLM\Audio_transcription_files\h.wav'
audio_array, sample_rate = librosa.load(audio_path, sr=None)

In [ ]:
model = AutoModel(model='paraformer-zh',kwargs={"disable_update": True, "device": "cpu"},punc_model ="ct-punc", vad_model="fsmn-vad", 
                                                     vad_kwargs={"max_single_segment_time": 30000},spk_model="cam++", 
                                                     spk_model_revision="v2.0.2")


# en
res = model.generate(
    input=audio_16KHz,
    cache={},
    language="zn",
    use_itn=True,
    batch_size_s=60,
    merge_vad=True,  #
    merge_length_s=15,
)
text = rich_transcription_postprocess(res[0]["text"])
print(text)



# ct-punc

In [ ]:
# This model is used to detect the punctuation in the generated text
model = AutoModel(model="ct-punc", model_revision="v2.0.4")
res = model.generate(input="那今天的会就到这里吧 happy new year 明年见")
print(res)

# fsmn-vad

In [ ]:
from funasr import AutoModel

model = AutoModel(model="fsmn-vad", model_revision="v2.0.4")

res = model.generate(input=audio_16KHz)
print(res)

# Combined all

In [ ]:
model = AutoModel(
    model='iic/SenseVoiceSmall',
     vad_model="fsmn-vad",
     vad_kwargs={"max_single_segment_time": 30000},
    device="cpu",
    #punc_model = "ct-punc"
)

# en
res = model.generate(
    input=audio_16KHz,
    cache={},
    language="zn",  # "zn", "en", "yue", "ja", "ko", "nospeech"
    use_itn=True,
    batch_size_s=60,
    merge_vad=True,  #
    merge_length_s=15,
)
text = rich_transcription_postprocess(res[0]["text"])
print(text)

In [ ]:
# Save the transcription to a text file
with open("transcription_SenseVoiceSmall-fsmn-vad.txt", "w", encoding="utf-8") as file:
    file.write(text)

# Intensity Checking

In [ ]:
import numpy as np
import librosa
import logging

def text_generator(model, audio_sample):
    text = model.generate(
        input=audio_sample,
        cache={},
        language="auto",
        use_itn=True,
        batch_size_s=60,
        merge_vad=False
    )
    return text
    
def compute_intensity(segment):
    """Compute the intensity of a given audio segment as RMS."""
    return np.sqrt(np.mean(segment**2))

def filter_high_intensity_segments(audio_array, segment_duration: float, intensity_threshold: float):
    """Filter high-intensity audio segments from the given audio array."""
    segment_samples = int(segment_duration * 16000)  # Number of samples per segment
    high_intensity_segments = []

    # Split audio into segments and process their intensities
    num_segments = len(audio_array) // segment_samples
    for i in range(num_segments):
        start_sample = i * segment_samples
        end_sample = start_sample + segment_samples
        segment = audio_array[start_sample:end_sample]
        
        # Calculate intensity for each segment
        intensity = compute_intensity(segment)
        print(f"Segment {i+1} Intensity: {intensity:.6f}")
        
        # Only keep the segment if its intensity exceeds the threshold
        if intensity > intensity_threshold:
            logging.info(f"Keeping high-intensity segment {i+1} with intensity {intensity:.6f}")
            high_intensity_segments.append(segment)
        else:
            logging.info(f"Skipping low-intensity segment {i+1} with intensity {intensity:.6f}")

    # If there are remaining samples that don't form a full segment, calculate their intensity
    if len(audio_array) % segment_samples != 0:
        segment = audio_array[num_segments * segment_samples:]
        intensity = compute_intensity(segment)
        print(f"Last Segment Intensity: {intensity:.6f}")
        if intensity > intensity_threshold:
            logging.info(f"Keeping last high-intensity segment with intensity {intensity:.6f}")
            high_intensity_segments.append(segment)

    return high_intensity_segments

def audio_to_text_model_offline(audio_file_path: str, model, intensity_threshold: float = 0.00) -> str:
    """Converts audio file to text using the provided model, processing only high-intensity segments."""
    try:
        # Load the audio file
        audio_array, sampling_rate = librosa.load(audio_file_path, sr=None)
        audio_array = librosa.resample(audio_array, orig_sr=sampling_rate, target_sr=16000)

        # Define the duration of each segment (for example, 2 seconds)
        segment_duration = 2.0  # seconds

        # Filter high-intensity segments
        high_intensity_segments = filter_high_intensity_segments(audio_array, segment_duration, intensity_threshold)

        # Concatenate high-intensity segments into a single array
        if high_intensity_segments:
            combined_audio = np.concatenate(high_intensity_segments)
            logging.info("Transcribing combined audio of length: %d", len(combined_audio))
            print(combined_audio)

            # Check if the length of the combined audio is greater than 30 seconds
            if len(combined_audio) > (30 * 16000):
                transcriptions = []
                sample_length = int(30 * 16000)
                for start in range(0, len(combined_audio), sample_length):
                    end = min(start + sample_length, len(combined_audio))
                    chunk = combined_audio[start:end]
                    generated_text = text_generator(model, chunk)
                    transcriptions.append(generated_text[0]["text"])
                
                # Join all transcriptions
                transcription_result = " ".join(transcriptions)
            else:
                # Transcribe the combined audio
                res = model.generate(
                    input=combined_audio,
                    vad_model="fsmn-vad",
                    cache={},
                    language="auto",
                    use_itn=True,
                    batch_size_s=60,
                    merge_vad=False
                )
                transcription_result = generated_text[0]["text"]
        else:
            logging.info("No high-intensity segments to transcribe.")
            transcription_result = ""

        return transcription_result.strip()
    
    except Exception as e:
        logging.error("Error in audio_to_text_model_offline: %s", e)
        return ""


In [ ]:
from funasr import AutoModel
path = r'C:\Users\Administrator\Desktop\Backend-Algorithm-LLM\Algorithm\audio_text_llm\SenseVoiceSmall\audio_sample\How to Learn Chinese Faster and Smarter.mp3'

model = AutoModel(
    model='paraformer-zh',  # This is the directory of model path
    device="cuda:0",  # "cuda:0" for GPU (if CUDA is available) or "cpu" for CPU.
    hub="hf",   # "hf" for Hugging Face Hub, "local" for local filesystem.
)


transcription_result = audio_to_text_model_offline(path, model)
print(transcription_result)


In [ ]:
transcription_result

In [ ]:
import sounddevice as sd

# Get the default input device details
device_info = sd.query_devices(sd.default.device[0], 'input')

# Print the sample rate of the input device
print(f"Device Sampling Rate: {device_info['default_samplerate']} Hz")


In [ ]:
import numpy as np
import sounddevice as sd

# Set the sampling rate
sampling_rate = 44100  # Sample rate in Hz

def audio_callback(indata, frames, time, status):
    """Callback function to process the audio data."""
    if status:
        print(status)
    # Convert the input data to a 1D array
    audio_data = indata[:, 0]  # Assuming mono audio
    # Calculate intensity as the root mean square (RMS) of the audio samples
    intensity = np.sqrt(np.mean(audio_data**2))  # RMS calculation
    print(f'Intensity: {intensity*1000:.6f}')

# Start recording from the microphone
with sd.InputStream(callback=audio_callback, channels=1, samplerate=sampling_rate):
    print("Recording... Press Ctrl+C to stop.")
    try:
        # Keep the program running indefinitely
        while True:
            pass
    except KeyboardInterrupt:
        print("Stopped recording.")


In [ ]:
import numpy as np
import librosa
import soundfile as sf

# Function to filter the audio based on intensity (e.g., removing low-intensity parts)
def filter_audio_by_intensity(audio_data, threshold, frame_length=2048, hop_length=512):
    """
    Filter audio based on RMS intensity for each frame using librosa's split method.
    
    Parameters:
        audio_data: The audio data array (1D).
        threshold: The intensity threshold to keep audio (scaled to original RMS).
        frame_length: The number of samples in each frame (default 2048 for efficiency).
        hop_length: The number of samples to shift between consecutive frames.
    """
    # Use librosa to split the audio based on energy threshold (this works similarly to RMS-based intensity filtering)
    # This split will return non-silent segments as start and end indices
    intervals = librosa.effects.split(audio_data, top_db=threshold)

    filtered_audio = []
    
    # Iterate over the intervals (start and end) and append the corresponding segments to filtered_audio
    for start, end in intervals:
        filtered_audio.append(audio_data[start:end])
    
    # Concatenate all non-silent segments into one array
    filtered_audio = np.concatenate(filtered_audio)
    
    return filtered_audio

# Load, filter based on intensity, and save the audio
def process_audio(file_path, intensity_threshold=60, output_file="output_filtered.wav"):
    # Load the audio file using librosa (it automatically resamples to 22050 Hz by default)
    audio_data, sampling_rate = librosa.load(file_path, sr=None)  # sr=None keeps the original sampling rate
    print(f"Original Sampling Rate: {sampling_rate} Hz")
    
    # Filter the audio based on intensity threshold (using split to detect non-silent segments)
    filtered_audio = filter_audio_by_intensity(audio_data, intensity_threshold)
    
    # Check if any audio remains after filtering
    if filtered_audio.size > 0:
        # Save the filtered audio to a new file using soundfile
        sf.write(output_file, filtered_audio, sampling_rate)
        print(f"Filtered audio saved to {output_file}")
    else:
        print("No audio passed the intensity threshold.")

# Example usage
file_path = r'C:\Users\Administrator\Desktop\Backend-Algorithm-LLM\Algorithm\audio_text_llm\SenseVoiceSmall\output_audio.wav'  # Replace with the path to your audio file
intensity_threshold = 30  # Threshold for filtering (in terms of RMS, for split method it's in dB)
process_audio(file_path, intensity_threshold)


In [ ]:
import sounddevice as sd
import numpy as np
import soundfile as sf

# Function to record 1 minute of audio
def record_audio(duration=60, output_file='output_audio.wav', sample_rate=44100, channels=1):
    """
    Record audio for a given duration and save it as a WAV file.
    
    :param duration: The duration of the recording in seconds (default is 60 seconds).
    :param output_file: The name of the file to save the audio.
    :param sample_rate: The sample rate of the recording (default is 44100).
    :param channels: The number of audio channels (default is 1 for mono).
    """
    print(f"Recording for {duration} seconds...")

    # Record audio using sounddevice
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=channels, dtype='int16')
    sd.wait()  # Wait until recording is finished
    
    # Save the recorded audio to a file using SoundFile
    sf.write(output_file, audio_data, sample_rate)
    print(f"Audio saved to {output_file}")

# Call the function to record for 1 minute (60 seconds)
record_audio(duration=60, output_file='output_audio.wav')


In [ ]:
pip install noisereduce 

In [11]:
import librosa
import noisereduce as nr

# Load the audio file
audio_data, sampling_rate = librosa.load(r'C:\Users\Administrator\Desktop\Backend-Algorithm-LLM\Algorithm\audio_text_llm\SenseVoiceSmall\output_audio.wav', sr=None)

# Apply noise reduction
reduced_noise_audio = nr.reduce_noise(
    y=audio_data,
    sr=44100,
    prop_decrease=1,  # Reduce noise by 50%
    thresh_n_mult_nonstationary=3,  # Moderate noise threshold for non-stationary noise
    n_std_thresh_stationary=50,  # Higher threshold for stationary noise
    time_constant_s=50,  # Longer adaptation for smoother noise reduction
    clip_noise_stationary=True,  # Clip stationary noise
    use_torch=False,  # Use CPU-based method (if True, would use GPU if available)
    device="cuda",  # Only relevant if use_torch=True
)

# Save the cleaned audio
sf.write('cleaned_audio.wav', reduced_noise_audio, sampling_rate)


In [23]:
import numpy as np
import soundfile as sf

# Function to calculate the RMS intensity of the audio
def calculate_intensity(audio_data):
    # Calculate intensity as the root mean square (RMS) for the entire audio
    return np.mean(np.abs(audio_data))

# Function to manually split the audio into frames
def split_audio_into_frames(audio_data, frame_length, hop_length):
    frames = []
    for start in range(0, len(audio_data) - frame_length + 1, hop_length):
        frame = audio_data[start:start + frame_length]
        frames.append(frame)
    return frames

# Function to filter the audio based on intensity (e.g., removing low-intensity parts)
def filter_audio_by_intensity(audio_data, threshold, frame_length=22050, hop_length=22050):
    """
    Filter audio based on RMS intensity for each frame.
    
    Parameters:
        audio_data: The audio data array (1D).
        threshold: The intensity threshold to keep audio (scaled to original RMS).
        frame_length: The number of samples in each frame (default 1 second if sr=22050).
        hop_length: The number of samples to shift between consecutive frames.
    """
    # Split the audio data into frames manually (no overlap)
    frames = split_audio_into_frames(audio_data, frame_length, hop_length)
    
    # Initialize a list to hold the filtered frames
    filtered_audio = []

    # Iterate over each frame and calculate its RMS
    for i, frame in enumerate(frames):
        rms = np.mean(np.abs(frame))

        print(f"Frame {i+1} RMS: {rms*1000:.6f}")  # Print the RMS value of each frame
        
        if rms > threshold:  # Only keep frames above the threshold
            filtered_audio.extend(frame)
    
    # Convert the filtered list back into a numpy array
    return np.array(filtered_audio)

# Load, filter based on intensity, and save the audio
def process_audio(file_path, intensity_threshold=0.05, output_file="output_filtered.wav"):
    # Load the audio file using soundfile (to keep the original sampling rate)
    audio_data, sampling_rate = sf.read(file_path)  # sf.read automatically detects the sampling rate
    print(f"Original Sampling Rate: {sampling_rate} Hz")
    
    # Calculate and print the original intensity (RMS)
    original_intensity = calculate_intensity(audio_data)
    print(f"Original Intensity (RMS): {original_intensity*1000:.6f}")
    
    # Adjust the threshold by dividing it by 1000 to match the unamplified scale
    intensity_threshold = intensity_threshold / 1000  # Adjust threshold based on your scaling
    
    # Filter the audio based on intensity threshold
    filtered_audio = filter_audio_by_intensity(audio_data, intensity_threshold)
    
    # Check if any audio remains after filtering
    if filtered_audio.size > 0:
        # Save the filtered audio to a new file using soundfile
        sf.write(output_file, filtered_audio, sampling_rate)
        print(f"Filtered audio saved to {output_file}")
    else:
        print("No audio passed the intensity threshold.")

# Example usage
file_path = r'C:\Users\Administrator\Desktop\Backend-Algorithm-LLM\Algorithm\audio_text_llm\SenseVoiceSmall\output_audio.wav'  # Replace with the path to your audio file
intensity_threshold = 3 # Threshold for filtering (in terms of RMS)
process_audio(file_path, intensity_threshold)


Original Sampling Rate: 44100 Hz
Original Intensity (RMS): 4.663745
Frame 1 RMS: 0.925991
Frame 2 RMS: 1.733966
Frame 3 RMS: 0.939083
Frame 4 RMS: 0.354866
Frame 5 RMS: 8.948013
Frame 6 RMS: 11.367237
Frame 7 RMS: 11.611264
Frame 8 RMS: 10.260269
Frame 9 RMS: 10.627591
Frame 10 RMS: 5.603942
Frame 11 RMS: 3.864626
Frame 12 RMS: 4.497665
Frame 13 RMS: 0.619096
Frame 14 RMS: 1.984582
Frame 15 RMS: 3.468934
Frame 16 RMS: 4.607754
Frame 17 RMS: 0.176029
Frame 18 RMS: 8.023485
Frame 19 RMS: 8.450524
Frame 20 RMS: 2.252986
Frame 21 RMS: 2.176846
Frame 22 RMS: 5.675308
Frame 23 RMS: 3.541254
Frame 24 RMS: 5.636208
Frame 25 RMS: 3.040562
Frame 26 RMS: 0.511544
Frame 27 RMS: 0.543065
Frame 28 RMS: 5.096907
Frame 29 RMS: 5.273408
Frame 30 RMS: 7.054164
Frame 31 RMS: 0.352844
Frame 32 RMS: 5.921265
Frame 33 RMS: 2.128933
Frame 34 RMS: 3.863606
Frame 35 RMS: 21.350397
Frame 36 RMS: 11.587578
Frame 37 RMS: 10.729396
Frame 38 RMS: 6.435291
Frame 39 RMS: 0.205064
Frame 40 RMS: 0.179457
Frame 41 RMS: 